## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

#imports the previous directory for config.py
import sys
sys.path.insert(1, '../')

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [4]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Current Description
0,0,Pevek,69.70,170.31,13.98,96,84,2.39,RU,2020-11-24 18:34:33,broken clouds
1,1,Kavieng,-2.57,150.80,83.23,77,100,12.50,PG,2020-11-24 18:34:33,overcast clouds
2,2,Nanning,22.82,108.32,60.80,87,40,6.71,CN,2020-11-24 18:30:33,scattered clouds
3,3,Carnarvon,-24.87,113.63,71.60,83,0,16.11,AU,2020-11-24 18:34:33,clear sky
4,4,Arraial Do Cabo,-22.97,-42.03,78.80,78,75,13.87,BR,2020-11-24 18:34:33,broken clouds


In [7]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
usr_min_temp = float(input('What is your minimum tolerated temperature?'))
usr_max_temp = float(input('What is your maximum tolerated temperature?'))

What is your minimum tolerated temperature?60
What is your maximum tolerated temperature?75


In [11]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
usr_filtered_df = city_data_df.loc[(city_data_df['Max Temp'] <= usr_max_temp)
                                   & (city_data_df['Max Temp'] >= usr_min_temp)]
usr_filtered_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Current Description
2,2,Nanning,22.82,108.32,60.80,87,40,6.71,CN,2020-11-24 18:30:33,scattered clouds
3,3,Carnarvon,-24.87,113.63,71.60,83,0,16.11,AU,2020-11-24 18:34:33,clear sky
5,5,East London,-33.02,27.91,66.20,55,0,13.87,ZA,2020-11-24 18:34:33,clear sky
11,11,Rikitea,-23.12,-134.97,74.03,78,100,12.46,PF,2020-11-24 18:34:33,overcast clouds
12,12,Flinders,-34.58,150.86,66.00,82,97,1.01,AU,2020-11-24 18:34:34,overcast clouds
...,...,...,...,...,...,...,...,...,...,...,...
674,674,Rafai,4.95,23.92,73.62,57,19,2.71,CF,2020-11-24 18:35:21,few clouds
676,676,Ocampo,21.65,-101.50,70.88,26,1,8.01,MX,2020-11-24 18:35:21,clear sky
680,680,Benjamin Hill,30.17,-111.10,72.00,31,0,1.01,MX,2020-11-24 18:35:22,clear sky
683,683,Odweyne,9.41,45.06,68.50,81,51,5.39,SO,2020-11-24 18:35:22,broken clouds


In [16]:
# 4a. Determine if there are any empty rows.
usr_filtered_df.isna().sum()

# there are two countries with null values

Unnamed: 0             0
City                   0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Country                2
Date                   0
Current Description    0
dtype: int64

In [21]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = usr_filtered_df.dropna()

In [22]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Nanning,CN,60.80,scattered clouds,22.82,108.32,
3,Carnarvon,AU,71.60,clear sky,-24.87,113.63,
5,East London,ZA,66.20,clear sky,-33.02,27.91,
11,Rikitea,PF,74.03,overcast clouds,-23.12,-134.97,
12,Flinders,AU,66.00,overcast clouds,-34.58,150.86,
13,Buraydah,SA,66.20,clear sky,26.33,43.98,
16,Kutoarjo,ID,61.00,broken clouds,-7.72,109.91,
18,Cape Town,ZA,62.01,scattered clouds,-33.93,18.42,
25,Kidal,ML,69.67,clear sky,18.44,1.41,
32,Erzin,TR,61.00,few clouds,36.96,36.20,


In [29]:

# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    params["location"] = f"{lat},{lng}"

    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    
    except (IndexError):
        print("Hotel not found... skipping.")
        
hotel_df
    
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Nanning,CN,60.80,scattered clouds,22.82,108.32,Nanning Travelling With Hostel
3,Carnarvon,AU,71.60,clear sky,-24.87,113.63,Hospitality Carnarvon
5,East London,ZA,66.20,clear sky,-33.02,27.91,Tu Casa
11,Rikitea,PF,74.03,overcast clouds,-23.12,-134.97,Pension Maro'i
12,Flinders,AU,66.00,overcast clouds,-34.58,150.86,Shellharbour Resort & Conference Centre
...,...,...,...,...,...,...,...
674,Rafai,CF,73.62,few clouds,4.95,23.92,
676,Ocampo,MX,70.88,clear sky,21.65,-101.50,Hotel Estancia del Vaquero
680,Benjamin Hill,MX,72.00,clear sky,30.17,-111.10,Luis Guillermo García Medina
683,Odweyne,SO,68.50,broken clouds,9.41,45.06,Eng Hiirey House


In [57]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df = hotel_df.loc[(hotel_df['Hotel Name'] != '')]

clean_hotel_df = hotel_df.dropna()

clean_hotel_df.isnull().sum()

Unnamed: 0             0
City                   0
Country                0
Max Temp               0
Current Description    0
Lat                    0
Lng                    0
Hotel Name             0
dtype: int64

In [67]:
# 8a. Create the output File (CSV)
output_data_file = 'WeatherPy_vacation.csv'
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [64]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [2]:
# 11a. Add a marker layer for each city to the map. 

locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

NameError: name 'clean_hotel_df' is not defined